先以vgg19训练特征提取网络，输入为64 * 64的图像，输出为30的属性，最后输出为softmax，以mse为损失函数

数据分析

In [1]:
root_train_path = r'D:\softfiles\workspace\games\zhi_jiang\original_data\DatasetA_train_20180813'

In [3]:
import numpy as np
import pandas as pd
import os

加载attribute_list.txt

1. 加载attributes_per_class.txt

In [49]:
attrbutes_per_class = pd.read_csv(os.path.join(root_train_path, 'attributes_per_class.txt'), sep='\t', header=None, index_col=0)
# 设定列名
attrbutes_per_class.columns = attribute_list.iloc[:, 0]
attrbutes_per_class.head()

1,is animal,is transportation,is clothes,is plant,is tableware,is device,is black,is white,is blue,is brown,...,is for communication,is for family,is for office use,is for personal,is gorgeous,is simple,is elegant,is cute,is pure,is naive
0,,,,,,,,,,,,,,,,,,,,,
ZJL1,1,0.0,0,0,0,0,0.5,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0
ZJL10,1,0.0,0,0,0,0,0.5,0.0,0.0,0.7,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0
ZJL100,0,0.0,0,0,0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0,0,0.0,0,0
ZJL101,0,0.0,0,0,0,0,0.8,0.1,0.3,0.1,...,0.0,1.0,0.0,1.0,0.0,0,0,0.0,0,0
ZJL102,0,0.0,0,0,0,0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0,0,0.0,0,0


In [51]:
# 去除标注属性值最大值为0的列，即为无效列，去除四列
attrbutes_per_class = attrbutes_per_class.loc[:, attrbutes_per_class.max() != 0]
# attrbutes_per_class.head()

In [53]:
attrbutes_per_class.max()
# is brown 这一列的值为2，归一到（0，1）

1
is animal               1.0
is transportation       1.0
is clothes              1.0
is plant                1.0
is tableware            1.0
is device               1.0
is black                1.0
is white                1.0
is blue                 1.0
is brown                2.0
is orange               1.0
is red                  1.0
is green                1.0
is yellow               1.0
has feathers            1.0
has four legs           1.0
has two legs            1.0
has two arms            1.0
is for entertainment    1.0
is for business         1.0
is for communication    1.0
is for family           1.0
is for office use       1.0
is for personal         1.0
is gorgeous             0.5
is cute                 0.1
dtype: float64

In [61]:
attrbutes_per_class = attrbutes_per_class.clip_upper(1.0)

In [60]:
attrbutes_per_class.to_csv('data/attrbutes_per_class.txt')

## 数据集划分

In [ ]:
class myImageGenerator():
def myImageGenerator(filepath, batch_size):
    

# 使用vgg19进行特征提取

In [64]:
from keras.layers import Dense, Flatten, Embedding, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications import vgg19
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 190

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

base_model = vgg19.VGG19(include_top=False, weights=None, input_shape=(64, 64, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation='sigmoid', name='fc6')(x)
x = Dense(190, activation='sigmoid', name='fc7')(x)
model = Model(inputs=base_model.input, outputs=x)
# print(model.summary())
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

# 创建callbacks
save_dir = os.path.join('output', 'saved_models')
model_name = 'model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

ValueError: Unknown loss function:crossentropy